# MMSegmentation

Notebook used for Semantic Segmentation task using drone images, satellite images and various warped satellite images.


Reference: https://github.com/open-mmlab/mmsegmentation

## Installation and setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Make sure to have a semester project directory in your drive storage and ```cd``` there.

In [ ]:
%cd "/content/drive/MyDrive/SemesterProj"

### Install required libraries

Check that you are using a GPU for execution. Also check available GPU memory.

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

We use PyTorch 1.10 and CUDA 11.1. You may install other versions by change the version number in pip install command, but this is not advised.

In [ ]:
# Install PyTorch
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0

In [ ]:
# Remove existing mmsegmentation and install new mmsegmentation library
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Check mmseg and torch versions installed.

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

## Dataset

### Prepare own dataset format

**IMPORTANT NOTE:** You are now in the ```mmsegmentation``` folder which will be installed each time you run the ```Installation and Setup``` section. When your Colab session terminates or breaks this folder will be deleted as you would have to install all the libraries again, as they are depending on the current GPU provided. Therefore, **pay attention because you have to keep your data and saved models or files out of this directory!** For this reason our file structure looks like this:
```
/content/drive/MyDrive/SemesterProj
├── data
│   ├── 120m                     <- dataset folder for this notebook
│   │   ├── ann_dir              <- satellite labels
|   |   |   ├── <img_name_1>.png     
│   │   │   └── ...
│   │   ├── img_dir              <- satellite images
|   |   |   ├── <img_name_1>.jpg    
│   │   │   └── ...
│   │   ├── human_drone_ann_dir  <- drone labels
│   │   ├── drone_dir            <- drone images
│   │   ├── splits               <- this folder will be created by code
│   │   │   ├── train.txt        <- train images
│   │   │   └── val.txt          <- val images
│   │   ├── results1             <- other folders with results from predictions or warping
|   |   |   ├── <img_name_1>.png   
│   │   │   └── ...
│   │   └── ...                  <- other folders
│   └── ...                      <- other dataset folder
├── mmsegmentation               <- previously installed folder for mmsegmentation repo       <- YOU ARE HERE!
│   ├── data                     <- symlink to external data folder !!!!!
│   ├── mmseg                    <- subfolders of mmsegmentation
│   ├── configs                  <- configs folder from mmsegmentation
│   ├── checkpoints              <- checkpoints folder from mmsegmentation
│   └── ...                      <- subfolders of mmsegmentation
└── ...
```
**NOTE:** You should still have a folder called ```data``` under ```mmsegmentation``` that will point to the external ```data``` folder. We will create a symlink later!

### Declare variables and paths for dataset configuration and visualization

In [7]:
import os
import mmcv
import math
import numpy as np
import time
import tqdm
import cv2
from PIL import Image
import os.path as osp
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from mmcv import Config
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

Create symlink to external data folder (see explanation in *Prepare own dataset format* section)

In [8]:
os.symlink('/content/drive/MyDrive/SemesterProj/data', 'data')

**Declare classes and palette**

In [9]:
# Class names used for training
classes = ('background', 'building', 'road', 'water')

# RGB color for each class
palette = [[0, 0, 0], [255, 0, 0], [0, 0, 255], [0, 255, 0]]

**Declare variables**

In [ ]:
# All data directory
data_dir = "data"

# Current dataset directory
dataset_dir = "120m"

# Training samples directory name
img_dir = "img_dir"

# Labels directory name
ann_dir = "ann_dir"

# Splits folder name
split_dir = 'splits'

# Model used in training or inference ('deeplabv3plus' or 'unet')
model_type = 'deeplabv3plus'

# Experiment id - for tracking experiments under the same configuration
exp_id = 0

# Custom image scale used in some experiments
custom_img_scale = (1056, 792)

data_root = os.path.join(data_dir, dataset_dir)
img_root = os.path.join(data_root, img_dir)
ann_root = os.path.join(data_root, ann_dir)

print(data_root)
print(img_root)
print(ann_root)

Look at one training image


In [ ]:
# Change image name according to your files structure
viz_image_name = 'DJI_0339_120'

img = mmcv.imread(os.path.join(img_root, viz_image_name + '.jpg'))
plt.figure(figsize=(8, 6))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

Look at one label

In [ ]:
img = Image.open(os.path.join(ann_root, viz_image_name + '.png'))
plt.figure(figsize=(8, 6))
im = plt.imshow(np.array(img.convert('RGB')))

# create a patch (proxy artist) for every color 
patches = [mpatches.Patch(color=np.array(palette[i])/255., 
                          label=classes[i]) for i in range(len(classes))]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., 
           fontsize='large')

plt.show()

Declare our own dataset subclass, here `SatelliteDataset`.

In [13]:
@DATASETS.register_module()
class SatelliteDataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, ignore_index, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, ignore_index=ignore_index, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

Set and create working directory.It will have the format: exp\<exp_id>_\<model_type>_\<img_dir>_\<label_dir>_\<split>

**IMPORTANT:** The working directory should be outside mmsegmentation folder.

In [ ]:
work_dir = '../exp{}_{}_{}_{}_{}'.format(exp_id, model_type, img_dir, ann_dir, split_dir)
mmcv.mkdir_or_exist(work_dir)
print(work_dir)

## Checkpoints downloading


Check https://github.com/open-mmlab/mmsegmentation/tree/master/configs for a specific pretrained net and a specific checkpoint file (.pth) and replace that filename in the ```wget``` command. We chose a pretrained DeepLabV3+ with a ResNet50 backbone, on CityScapes Dataset and a pretrained UNet+FCN on CityScapes Dataset.


In [15]:
!mkdir checkpoints

DeepLabV3+ checkpoint

In [ ]:
!wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_80k_cityscapes/deeplabv3plus_r50-d8_512x1024_80k_cityscapes_20200606_114049-f9fb496d.pth -P checkpoints

UNet checkpoint

In [ ]:
!wget https://download.openmmlab.com/mmsegmentation/v0.5/unet/fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes/fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes_20211210_145204-6860854e.pth -P checkpoints

In the next step, we need to modify the config for the training. To accelerate the process, we finetune the model from trained weights.

The path to your desired base configuration file should match your checkpoint file.

## Create config file

Choose base configuration file according to model used and modify according to your dataset. (you may change this function)

In [18]:
def create_config(model_type, classes, data_root, img_dir, ann_dir, split_dir, work_dir, custom_img_scale, resume_file=''):

    # Give the path to your desired base configuration file
    if model_type == 'deeplabv3plus':
        cfg = Config.fromfile('configs/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_80k_cityscapes.py')
    elif model_type == 'unet':
        cfg = Config.fromfile('configs/unet/fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes.py')

    # Since we use only one GPU, BN is used instead of SyncBN
    cfg.norm_cfg = dict(type='BN', requires_grad=True)
    cfg.model.backbone.norm_cfg = cfg.norm_cfg
    cfg.model.decode_head.norm_cfg = cfg.norm_cfg
    cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

    # Modify num classes of the model in decode/auxiliary head
    cfg.model.decode_head.num_classes = len(classes)
    cfg.model.auxiliary_head.num_classes = len(classes)

    # For UNet model we need to proceed in a sliding window fashion, when validating
    if model_type == 'unet':
        cfg.model.test_cfg.mode='slide'
        cfg.model.test_cfg.crop_size=(512, 1024)
        # Stride is defined according to image size and crop size
        cfg.model.test_cfg.stride=(492, 608)

    # Modify dataset type and path
    cfg.dataset_type = 'SatelliteDataset'
    cfg.data_root = data_root

    # *Important*: The default learning rate in config files is for 4 GPUs and 2 img/gpu (batch size = 4x2 = 8).
    cfg.data.samples_per_gpu = 2
    cfg.data.workers_per_gpu= 2

    # Cityscapes statistics
    cfg.img_norm_cfg = dict(
        mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

    cfg.crop_size = (1024, 512)
    cfg.train_pipeline = [
        dict(type='LoadImageFromFile'),
        dict(type='LoadAnnotations'),
        dict(type='Resize', img_scale=custom_img_scale, ratio_range=(0.5, 2.0)),
        dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
        dict(type='RandomFlip', flip_ratio=0.5),
        dict(type='PhotoMetricDistortion'),
        dict(type='Normalize', **cfg.img_norm_cfg),
        dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
        dict(type='DefaultFormatBundle'),
        dict(type='Collect', keys=['img', 'gt_semantic_seg']),
    ]

    cfg.test_pipeline = [
        dict(type='LoadImageFromFile'),
        dict(
            type='MultiScaleFlipAug',
            img_scale=custom_img_scale,
            # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
            flip=False,
            transforms=[
                dict(type='Resize', keep_ratio=True),
                dict(type='RandomFlip'),
                dict(type='Normalize', **cfg.img_norm_cfg),
                dict(type='ImageToTensor', keys=['img']),
                dict(type='Collect', keys=['img']),
            ])
    ]

    cfg.data.train.type = cfg.dataset_type
    cfg.data.train.data_root = cfg.data_root
    cfg.data.train.img_dir = img_dir
    cfg.data.train.ann_dir = ann_dir
    cfg.data.train.pipeline = cfg.train_pipeline
    cfg.data.train.split = split_dir + '/train.txt'
    cfg.data.train.ignore_index = 3

    cfg.data.val.type = cfg.dataset_type
    cfg.data.val.data_root = cfg.data_root
    cfg.data.val.img_dir = img_dir
    cfg.data.val.ann_dir = ann_dir
    cfg.data.val.pipeline = cfg.test_pipeline
    cfg.data.val.split = split_dir + '/val.txt'
    cfg.data.val.ignore_index = 3

    cfg.data.test.type = cfg.dataset_type
    cfg.data.test.data_root = cfg.data_root
    cfg.data.test.img_dir = img_dir
    cfg.data.test.ann_dir = ann_dir
    cfg.data.test.pipeline = cfg.test_pipeline
    cfg.data.test.split = split_dir + '/val.txt'
    cfg.data.test.ignore_index = 3

    if resume_file == '':
        # Load an existing model
        if model_type == 'deeplabv3plus':
            cfg.load_from = 'checkpoints/deeplabv3plus_r50-d8_512x1024_80k_cityscapes_20200606_114049-f9fb496d.pth'
        elif model_type == 'unet':
            cfg.load_from = 'checkpoints/fcn_unet_s5-d16_4x4_512x1024_160k_cityscapes_20211210_145204-6860854e.pth'
    else:
        # or resume from a specific checkpoint
        cfg.resume_from = os.path.join(work_dir, resume_file)

    # Set up working dir to save files and logs.
    cfg.work_dir = work_dir

    # Choose number of iterations according to dataset size and batch size
    cfg.runner.max_iters = 84000

    # Choose log interval(every epoch)
    cfg.log_config.interval = 210

    # Evaluation interval(every 5 epochs)
    cfg.evaluation.interval = 1050

    # Save model with the highest validation mIoU
    cfg.evaluation.save_best = 'mIoU'

    # Also, choose checkpoint interval to save model each 10 epochs
    cfg.checkpoint_config.interval = 2100

    # Set seed to facitate reproducing the result
    cfg.seed = 0
    set_random_seed(0, deterministic=False)
    cfg.gpu_ids = range(1)
    cfg.device = get_device()

    # Let's have a look at the final config used for training
    print(f'Config:\n{cfg.pretty_text}')

    return cfg

## Train and Evaluation

After setting the data, model checkpoints and configuration dictionary you can start training and evaluation.

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Define configuration
cfg = create_config(model_type, classes, data_root, img_dir, ann_dir, split_dir, work_dir, custom_img_scale, resume_file='iter_24990.pth')

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
print(cfg.work_dir)
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

## Inference

### Helper functions

In some cases of our experiments, we cannot use only the provided API for inference as we may need an inference function that does not rescale the output image to the original size (we want to keep the size to the custom defined one).

In [37]:
from mmseg.apis.inference import LoadImage
from mmseg.core.evaluation import get_palette
from mmcv.parallel import collate, scatter
from mmseg.datasets.pipelines import Compose
import warnings


def custom_inference_no_rescale(model, imgs):
    """Inference on custom image size"""

    cfg = model.cfg
    device = next(model.parameters()).device

    # build the data pipeline
    test_pipeline = [LoadImage()] + cfg.data.test.pipeline[1:]
    test_pipeline = Compose(test_pipeline)

    # prepare data
    data = []
    imgs = imgs if isinstance(imgs, list) else [imgs]
    for img in imgs:
        img_data = dict(img=img)
        img_data = test_pipeline(img_data)
        data.append(img_data)
    data = collate(data, samples_per_gpu=len(imgs))
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]
    else:
        data['img_metas'] = [i.data[0] for i in data['img_metas']]
        
    # forward the model
    with torch.no_grad():
        result = model(return_loss=False, rescale=False, **data)
    return result

def custom_result_pyplot(model, img, result, palette=None, fig_size=(15, 10),
                         opacity=0.5, title='', block=True, out_file=None):
    """Plot prediction on custom image size"""

    if hasattr(model, 'module'):
        model = model.module
    
    show = False
    img = mmcv.imread(img)
    img_data = dict(img=img)
    pipeline =  [dict(type='Resize', img_scale=[(1056, 792)], keep_ratio=False)]
    plot_pipeline = Compose(pipeline)
    img_data = plot_pipeline(img_data)
    img = np.array(img_data['img'])

    img = img.copy()
    seg = result[0]

    if palette is None:
        # Get random state before set seed,
        # and restore random state later.
        # It will prevent loss of randomness, as the palette
        # may be different in each iteration if not specified.
        # See: https://github.com/open-mmlab/mmdetection/issues/5844
        state = np.random.get_state()
        np.random.seed(42)
        # random palette
        palette = np.random.randint(
            0, 255, size=(len(classes), 3))
        np.random.set_state(state)
    else:
        palette = palette
        
    palette = np.array(palette)
    assert palette.shape[0] == len(classes)
    assert palette.shape[1] == 3
    assert len(palette.shape) == 2
    assert 0 < opacity <= 1.0
    color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
    for label, color in enumerate(palette):
        color_seg[seg == label, :] = color
    # convert to BGR
    color_seg = color_seg[..., ::-1]

    img = img * (1 - opacity) + color_seg * opacity
    img = img.astype(np.uint8)
    # if out_file specified, do not show image in window
    if out_file is not None:
        show = False

    if show:
        mmcv.imshow(img, '', 0)
    if out_file is not None:
        mmcv.imwrite(img, out_file)

    if not (show or out_file):
        warnings.warn('show==False and out_file is not specified, only '
                        'result image will be returned')
            
    plt.figure(figsize=fig_size)
    plt.imshow(mmcv.bgr2rgb(img))
    plt.title(title)
    plt.tight_layout()
    plt.show(block=block)
    if out_file is not None:
        mmcv.imwrite(img, out_file)


def get_train_filenames(data_root, split_dir, train=True, val=True):
    """Gathers all train and/or val data used"""

    splits_dir = os.path.join(data_root, split_dir)
    data_val = []
    data_train = []

    if val:
        with open(os.path.join(splits_dir, 'val.txt'), 'r') as f:
            data_val = f.readlines()
            data_val = [line[:-1] for line in data_val]
    if train:
        with open(os.path.join(splits_dir, 'train.txt'), 'r') as f:
            data_train = f.readlines()
            data_train = [line[:-1] for line in data_train]
    

    return sorted(data_train + data_val)


def prepare_for_inference(custom_image_scale, checkpoint_file, palette, cfg=None):
    """Prepares the configuration file for inference"""
    
    if cfg is None:
        raise ValueError('Config should be created first to pe prepared for inference')

    # Create custom test pipeline with resizing
    cfg.model.pretrained = None
    cfg.model.train_cfg = None
    cfg.load_from = None
    cfg.resume_from = None
    cfg.test_pipeline = [
        dict(type='LoadImageFromFile'),
        dict(type='Resize', img_scale=[custom_img_scale], keep_ratio=False),
        dict(
            type='MultiScaleFlipAug',
            img_scale=custom_img_scale,
            flip=False,
            transforms=[
                dict(type='Resize', keep_ratio=True),
                dict(type='RandomFlip'),
                dict(type='Normalize', **cfg.img_norm_cfg),
                dict(type='ImageToTensor', keys=['img']),
                dict(type='Collect', keys=['img']),
            ])
    ]
    cfg.data.test.pipeline = cfg.test_pipeline

    # Build the model from a config file and load 
    model = mmseg.models.build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
    checkpoint = mmcv.runner.load_checkpoint(model, checkpoint_file, map_location='cpu')
    model.CLASSES = checkpoint['meta']['CLASSES']
    model.PALETTE = palette
    model.cfg = cfg
    model.to('cuda:0')
    model.eval() 

    return model, cfg 


def get_best_checkpoint(work_dir):
    best_checkpoint = None
    files = sorted(os.listdir(work_dir))[::-1]
    for file in files:
        if 'best' in file:
            best_checkpoint = file
    if best_checkpoint is None:
        raise ValueError("No best checkpoints were saved, please select 1 manually")
    else:
        return os.path.join(work_dir, best_checkpoint)

### Inference check on one image from validation set

In [38]:
# Define an image to make inference on (here one image from validation set)
img_name = 'DJI_0328_120.jpg'
img = mmcv.imread(os.path.join(img_root, img_name))

In [ ]:
# Create initial config
cfg = create_config(model_type, classes, data_root, img_dir, ann_dir, split_dir, work_dir, custom_img_scale)

# Choose a checkpoint for loading the trained weights
checkpoint_file = get_best_checkpoint(work_dir)

# Update config and model for inference
model, cfg = prepare_for_inference(custom_img_scale, checkpoint_file, palette, cfg)

# Do inference and plot
result = inference_segmentor(model, img)
show_result_pyplot(model, img, result, palette)

## Inference on multiple drone images

### Setup

In [40]:
# If prediction should be on full image size or custom image size (if False)
fullsize = True

# Choose subset to make prediction on
train_img = False
val_img = True

# Define drone directory with images to make inference on:
drone_dir = 'drone_dir_full'

### Inference

**IMPORTANT**: Results file should be created outside mmsegmentation.

In [ ]:
# Define and create output folder for predictions
result_dir = 'prediction_{}_{}_{}_{}{}'.format(model_type, img_dir, ann_dir, split_dir, '_full_size' if fullsize else '_custom_size')
results_files_path = os.path.join(os.path.join('..', data_root), result_dir)
mmcv.mkdir_or_exist(results_files_path)

# Get all image names you want to make a prediction on
files = get_train_filenames(data_root, split_dir, train_img, val_img)
print("There are {} drone images".format(len(files)))

# Get directory with all drone images
drone_files_path = os.path.join(os.path.join('..', data_root), drone_dir)
print("Predicting on images from {}".format(drone_files_path))

# Create initial config
cfg = create_config(model_type, classes, data_root, img_dir, ann_dir, split_dir, work_dir, custom_img_scale)

# Choose a checkpoint for loading the trained weights
checkpoint_file = get_best_checkpoint(work_dir)

# Update config and model for inference
model, cfg = prepare_for_inference(custom_img_scale, checkpoint_file, palette, cfg)

for filename in files:
    img_path = os.path.join(drone_files_path, filename + '.jpg')
    img = mmcv.imread(img_path, backend='cv2')
    # We need a sleep function otherwise Colab will crash
    time.sleep(15)
    if fullsize:
        result = inference_segmentor(model, img)
    else:
        result = custom_inference_no_rescale(model, img)
    result = np.array(result).transpose((1, 2, 0)).squeeze().astype(np.uint8)

    # Convert prediction to 'P' mode
    seg_img = Image.fromarray(result).convert('P')
    seg_img.putpalette(np.array(palette, dtype=np.uint8))

    # Save results
    seg_img.save(os.path.join(results_files_path, filename.split('.')[0] + '.png'))
    print("{} done!".format(filename))

## Compute mIoU between ground truths and predictions

In [42]:
from mmseg.core.evaluation.metrics import mean_iou


def threshold_img(img_hsv):
    """Creates categorical label based on HSV colospace thresholding"""

    # define range of blue color in HSV
    lower_blue = np.array([115, 50, 50])
    upper_blue = np.array([125, 255, 255])
    mask_blue = cv2.inRange(img_hsv, lower_blue, upper_blue)

    # define range of green color in HSV
    lower_green = np.array([55, 50, 50])
    upper_green = np.array([65, 255, 255])
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)

    # lower mask (0-10)
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([5, 255, 255])
    mask0_red = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([175, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask1_red = cv2.inRange(img_hsv, lower_red, upper_red)

    # join my masks
    mask_red = mask0_red + mask1_red

    # set my output img to zero everywhere except my mask
    output_hsv = np.zeros((img_hsv.shape[0], img_hsv.shape[1]))
    output_hsv[np.where(mask_red == 255)] = 1
    output_hsv[np.where(mask_blue == 255)] = 2
    output_hsv[np.where(mask_green == 255)] = 3

    return output_hsv.astype(np.uint8)


def compute_mIoU_between_gt_and_pred(ann_path, pred_path, data_root, splits_dir, scale, fullsize=False):
    """
    Computes mIoU.
    If fullsize is True, scale must be also specified.
    """

    # Get all predicted (validation) filenames
    predicted_files = sorted(get_train_filenames(data_root, splits_dir, train=False, val=True))

    # Lists for gathering all images
    ann_labels = []
    pred_labels = []

    for filename in tqdm.tqdm(predicted_files):
        filename += '.png'
        if fullsize:
            ann_label = np.array(Image.open(os.path.join(ann_path, filename)))
        else:
            # Scale image according to 'scale' parameter
            label_ann = np.array(Image.open(os.path.join(ann_path, filename)).convert("RGB"))[:, :, ::-1]
            # time.sleep(15)
            label_ann = cv2.resize(label_ann, scale, interpolation=cv2.INTER_AREA)
            label_ann_hsv = cv2.cvtColor(label_ann, cv2.COLOR_BGR2HSV)
            ann_label = threshold_img(label_ann_hsv)
            
        ann_labels.append(ann_label)
        pred_label = np.array(Image.open(os.path.join(pred_path, filename)))
        # time.sleep(5)
        pred_labels.append(pred_label)
    
    # Compute mean IoU
    score_dict = mean_iou(pred_labels,
                        ann_labels,
                        num_classes=4,
                        ignore_index=-1,
                        nan_to_num=None,
                        label_map=dict(),
                        reduce_zero_label=False)
    
    # Detele unwanted variables and return result
    del ann_labels
    del pred_labels

    return score_dict

### mIoU between human drone labels and predicted drone labels (sat-sat, drone-sat, PatchMatch, LoFTR, etc.)

In [ ]:
# Choose human annotation directory
drone_label_path = "../data/120m/human_drone_ann_dir"

# Choose predictions directory
pred_path = '../data/120m/prediction_deeplabv3plus_img_dir_ann_dir_splits_custom_size'

score_dict = compute_mIoU_between_gt_and_pred(drone_label_path, pred_path, data_root,
                                              split_dir, scale=custom_img_scale, fullsize='full_size' in pred_path)
print(score_dict)

## Plot train loss and validation mIoU for a specific model


In [ ]:
import json

log_path = os.path.join(work_dir, 'None.log.json')
train_loss = []
val_miou = []

with open(log_path, 'r') as f:
    for line in f.readlines():
        data_dict = json.loads(line)
        if 'hook_msgs' in data_dict.keys():
            continue
        else:
            if data_dict['mode'] == 'train':
                train_loss.append(data_dict['loss'])
            elif data_dict['mode'] == 'val':
                val_miou.append(data_dict['mIoU'])

In [ ]:
plt.plot(train_loss)

In [ ]:
plt.plot(val_miou)